In [ ]:
# default_exp image.pillow_tools

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
import sys
import argparse
import logging
import base64
from io import BytesIO
from PIL import Image as PILImage
from PIL.ExifTags import TAGS as EXIF_TAGS
from mlcore.image.tools import ImageOrientation
from mlcore.io.core import scan_files

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

EXIF_ORIENTATION_TAG = 'Orientation'
"""The Image EXIF orientation tag"""

In [ ]:
# export

logger = logging.getLogger(__name__)

# Image Tools for Pillow
> Tools for converting images by using Pillow library.

In [ ]:
# export


def limit_to_max_size(img, max_size):
    """
    Limit the image size to max size and scale the image,
    if max size exceeded.
    `img`: The image to validate as Pillow Image.
    `max_size`: The max allowed image size.
    :return: The eventually resized image.
    """
    biggest_size = max(img.size)
    if max_size and biggest_size > max_size:
        ratio = 1.0 * max_size / biggest_size
        img = img.resize([int(ratio * s) for s in img.size])
    return img

In [ ]:
# export


def fit_to_max_size(img, max_width, max_height):
    """
    Limit the image size to maximum width and height and scale the image,
    if size exceeded.
    `img`: The image to validate as Pillow Image.
    `max_width`: The max allowed image width.
    `max_height`: The max allowed image height.
    :return: The eventually resized image.
    """
    w, h = img.size
    scale_delta = max(w - max_width, h - max_height)
    if scale_delta > 0:
        max_size = max(w - scale_delta, h - scale_delta)
        img = limit_to_max_size(img, max_size)
    return img

In [ ]:
# export


def get_image_size(fname):
    """
    Calculates image size of a given image file path.
    `fname`: the file path
    return: the image width and height
    """
    img = PILImage.open(fname)
    w, h = img.size
    return w, h

In [ ]:
# export


def get_image_size_and_orientation(fname):
    """
    Parses the EXIF orientation information from the image.
    :param img_bytes: The image data to extract the EXIF information from.
    :return: The image size and the orientation of the image.
    """
    orientation = ImageOrientation.TOP
    image = PILImage.open(fname)
    size = image.size
    if image is not None:
        exif_data = image._getexif()
        if exif_data is not None:
            for (k, v) in exif_data.items():
                if EXIF_TAGS.get(k) == EXIF_ORIENTATION_TAG:
                    try:
                        orientation = ImageOrientation(v)
                    except ValueError as e:
                        logger.error(e)
    return size, orientation

In [ ]:
# export


def convert_to_base64(image, image_type="PNG"):
    """
    Converts the specified image into a base64 version of itself.

    `image`: The image to transform as Pillow Image.
    `image_type`: The image type.
    :return: The base64 encoded version of the image.
    """
    buffered = BytesIO()
    image.save(buffered, format=image_type)
    return base64.b64encode(buffered.getvalue()).decode('UTF-8')


## Helper Methods

In [ ]:
# export


def configure_logging(logging_level=logging.INFO):
    """
    Configures logging for the system.

    :param logging_level: The logging level to use.
    """
    logger.setLevel(logging_level)

    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging_level)

    logger.addHandler(handler)

## Run from command line

To run the pillow image tools from command line, use the following command:
`python -m mlcore.image.pillow_tools [parameters]`

The following parameters are supported:
- `[image_path]`: The path to the image files.

In [ ]:
# export


if __name__ == '__main__' and '__file__' in globals():
    # for direct shell execution
    configure_logging()

    parser = argparse.ArgumentParser()
    parser.add_argument("image_path",
                        help="The path to the image files.")

    args = parser.parse_args()
    files = scan_files(args.image_path)
    for file in files:
        size, orientation = get_image_size_and_orientation(file)
        logger.info("Size: width: {}, height: {}, orientation: {}".format(size[0], size[1], orientation))
